In [1]:
import pandas as pd
import os

import plotly.express as px
import time

In [2]:
# labelalias = {
#     'constant-0': 'constant-0',
#     'Tanimoto-distance': 'Tanimoto',
#     'Tanimoto-distance-TIMES10': 'Tanimoto_times10',
#     'Tanimoto-distance-TIMES100': 'Tanimoto_times100',
#     'Tanimoto-distance-EXP': 'Tanimoto_exp',
#     'Tanimoto-distance-SQRT': 'Tanimoto_sqrt',
#     "Tanimoto-distance-NUM_NEIGHBORS_TO_1": "Tanimoto_nn_to_1",
# }
from value_functions import labelalias


In [8]:
eventid= '202305-2911-2320-5a95df0e-3008-4ebe-acd8-ecb3b50607c7'
input_folder = f"Runs/{eventid}"

output_folder = f'Plots/{eventid}/Cost_value_correlation_v2'

if not os.path.exists(output_folder):
        os.makedirs(output_folder)


# df_results_tot = pd.read_csv(f'{input_folder}/paroutes_n5_result_added_value_fns.csv')
df_results_tot = pd.read_csv(f'{input_folder}/paroutes_result_added_value_fns_v2.csv')


In [9]:
df_results_tot['is_solved'] = (df_results_tot['first_soln_time']!=-1 )
# df_results_tot['Embedding-from-fingerprints_TIMES10'] = 10.0 * df_results_tot['Embedding-from-fingerprints']
df_results_tot

,smiles,n_iter,first_soln_time,lowest_cost_route_found,lowest_cost_route_(binned),best_route_cost_lower_bound,lowest_depth_route_found,best_route_depth_lower_bound,num_calls_rxn_model,num_nodes_in_tree,is_purchasable,constant-0,Tanimoto-distance,Embedding-from-fingerprints,Embedding-from-gnn,is_solved
0,COc1ccc(F)c(-c2ccc(COc3cc(C(CC(=O)O)C4CC4)ccc3...,9999,-1.0,-1.000000,NotSolved,11.654802,-1.0,2.0,382,72602,0.0,0.0,0.641892,0.276396,1.0,False
1,CCc1cc2nncc(N3CCc4[nH]nc(C(=O)NC5CC5)c4C3)c2cc1OC,296,61.0,10.257000,009-011,9.335632,6.0,2.0,61,4129,0.0,0.0,0.677686,0.188257,1.0,True
2,Clc1ccc(-n2cncn2)c(Cc2nc3c(NCCC4CCCCN4)nccc3o2)c1,9999,357.0,29.260213,029-031,16.640865,10.0,2.0,668,69684,0.0,0.0,0.748148,0.221315,1.0,True
3,CCOCc1nc2c(N)nc3ccc(O)cc3c2n1CC(C)(C)NS(C)(=O)=O,11,6.0,2.447898,001-003,2.031529,6.0,1.0,6,287,0.0,0.0,0.698413,0.222840,1.0,True
4,O=C(O)CCNC(=O)c1nc(-c2ccncc2)c2c(cc(-c3ccccc3)...,15,5.0,0.876074,000-001,0.876074,5.0,1.0,8,341,0.0,0.0,0.731707,0.212420,1.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,CNC(=Nc1cc(O)cc(C(=O)O)c1)SC,1323,210.0,15.194417,014-016,11.411925,5.0,2.0,212,20823,0.0,0.0,0.606557,0.218558,1.0,True
9996,CC(C)(C)OC(=O)N1CC2CC(C1)c1cc(N)c(N)cc12,13,3.0,4.664657,004-006,4.149135,2.0,1.0,6,252,0.0,0.0,0.576087,0.096431,1.0,True
9997,Cc1ncc(C)n2nc(CCc3ccc(O)c(NC(C)CCO)n3)nc12,5144,529.0,16.362677,016-018,13.203531,5.0,2.0,529,47697,0.0,0.0,0.462264,0.213056,1.0,True
9998,COC(=O)c1cc(C)c(-c2ccc(OC)c(B3OC(C)(C)C(C)(C)O...,11,6.0,2.956156,001-003,2.956156,3.0,1.0,6,463,0.0,0.0,0.571429,0.148253,1.0,True


In [5]:
df_results_tot['Embedding-from-gnn'].unique()

array([1.])

In [6]:
values_to_consider = {
    "Tanimoto-distance": "Tanimoto",
    "Embedding-from-fingerprints": "Fnps-nn",
    "Embedding-from-gnn": "Gnn"
}

In [7]:
def plot_correlation(df_results_tot, col_name, col_label):
    fig = px.violin(df_results_tot, x="lowest_depth_route_found", y=col_name, width=1000, height=600,
    #              points='all',
                    box=True,
                    color='is_solved',
                    color_discrete_sequence = [px.colors.qualitative.Plotly[1], px.colors.qualitative.Plotly[2]],
                    violinmode='overlay',
                 labels={
    #                      "algorithm": None,
    #                      "value": "Time to first solution",
    #                      "species": "Species of Iris"
                     },
    #              title="Time to first solution"
                )
    fig.update_layout(xaxis_title="Cost of minimal-cost route to syntesise target </br></br> (assignig cost=1 to each reaction and cost=0 to each molecule in route)", 
                      yaxis_title=f"{col_label} distance between taget </br></br> and closest purchasable molecule",
                      title="Correlation between nearest neighbour distance and minimal cost route"
                     )
    # fig.update_xaxes(labelalias=labelalias, categoryorder='array', categoryarray=list(labelalias.keys()))
    # fig.write_image(f'{output_folder}/Boxplot_time_first_solution.png') 
    fig.write_image(f"{output_folder}/cost_value_correlation_{col_label}.pdf") 
    time.sleep(10)
    fig.write_image(f"{output_folder}/cost_value_correlation_{col_label}.pdf") 
    fig.show()

In [ ]:
for col_name, col_label in values_to_consider.items():
    plot_correlation(df_results_tot=df_results_tot, col_name=col_name, col_label=col_label)
    
    

In [ ]:
# fig = px.violin(df_results_tot, x="lowest_depth_route_found", y="Embedding-from-fingerprints", width=1000, height=600,
# #              points='all',
#                 box=True,
#                 color='is_solved',
#                 color_discrete_sequence = [px.colors.qualitative.Plotly[1], px.colors.qualitative.Plotly[0]],
#                 violinmode='overlay',
#              labels={
# #                      "algorithm": None,
# #                      "value": "Time to first solution",
# #                      "species": "Species of Iris"
#                  },
# #              title="Time to first solution"
#             )
# fig.update_layout(xaxis_title="Cost of minimal-cost route to syntesise target </br></br> (assignig cost=1 to each reaction and cost=0 to each molecule in route)", 
#                   yaxis_title="Fnps-nn distance between taget </br></br> and closest purchasable molecule",
#                   title="Correlation between nearest neighbour distance and minimal cost route"
#                  )
# # fig.update_xaxes(labelalias=labelalias, categoryorder='array', categoryarray=list(labelalias.keys()))
# # fig.write_image(f'{output_folder}/Boxplot_time_first_solution.png') 
# fig.write_image(f"{output_folder}/cost_value_correlation_Fnps-nn.pdf") 
# time.sleep(10)
# fig.write_image(f"{output_folder}/cost_value_correlation_Fnps-nn.pdf")
# fig.show()

In [ ]:
# fig = px.violin(df_results_tot, x="lowest_depth_route_found", y="Embedding-from-fingerprints_TIMES10", width=1000, height=600,
# #              points='all',
#                 box=True,
#                 color='is_solved',
#                 color_discrete_sequence = [px.colors.qualitative.Plotly[1], px.colors.qualitative.Plotly[0]],
#                 violinmode='overlay',
#              labels={
# #                      "algorithm": None,
# #                      "value": "Time to first solution",
# #                      "species": "Species of Iris"
#                  },
# #              title="Time to first solution"
#             )
# fig.update_layout(xaxis_title="Cost of minimal-cost route to syntesise target </br></br> (assignig cost=1 to each reaction and cost=0 to each molecule in route)", 
#                   yaxis_title="10 * Fnps-nn distance between taget </br></br> and closest purchasable molecule",
#                   title="Correlation between nearest neighbour distance and minimal cost route"
#                  )
# # fig.update_xaxes(labelalias=labelalias, categoryorder='array', categoryarray=list(labelalias.keys()))
# # fig.write_image(f'{output_folder}/Boxplot_time_first_solution.png') 
# fig.write_image(f"{output_folder}/cost_value_correlation_Fnps-nn_times10.pdf") 
# time.sleep(10)
# fig.write_image(f"{output_folder}/cost_value_correlation_Fnps-nn_times10.pdf") 
# fig.show()

In [ ]:
# fig = px.box(df_results_tot, x="lowest_depth_route_found", y="Tanimoto-distance", width=1000, height=600,
#              points='all',
#              labels={
# #                      "algorithm": None,
# #                      "value": "Time to first solution",
# #                      "species": "Species of Iris"
#                  },
# #              title="Time to first solution"
#             )
# # fig.update_layout(xaxis_title=None)
# # fig.update_xaxes(labelalias=labelalias, categoryorder='array', categoryarray=list(labelalias.keys()))
# # fig.write_image(f'{output_folder}/Boxplot_time_first_solution.png') 
# fig.show()

In [ ]:

# fig = px.violin(df_results_tot, x="lowest_cost_route_(binned)", y="Tanimoto-distance", width=1000, height=600,
#              box=True,
#                 labels={
# #                      "algorithm": None,
# #                      "value": "Time to first solution",
# #                      "species": "Species of Iris"
#                  },
# #              title="Time to first solution"
#             )
# fig.update_layout(xaxis_title=None)
# fig.update_xaxes(categoryorder="category ascending")
# # fig.update_xaxes(labelalias=labelalias, categoryorder='array', categoryarray=list(labelalias.keys()))


# fig.show()